## fMRIprep ANALYSIS FOR WELSH ADVANCED NEUROIMAGING DATABASE

This code will run fMRIprep using the CUBRIC computing cluster and is set up to work on data from the WAND project. Important scripts you need to be aware of are: </br>
- `fmriprep_cubric.py` in the `cnbpy/cnbpy` folder: this script contains the python code that is used to set up the analysis and send the job to the cluster
- `template_cubric.sh` in the `cnbpy/cnbpy/test/data` folder: this is the SLURM script that will tell the cluster what job to run
- `fmriprep_config_cubric.yml` in the `cnbpy/cnbpy/test/data` folder: this script contains the dictionary that is used to populate the SLURM script
</br>

At the moment, this script will run all participants in the directory, but I will write another one to do one at a time using an array.


### Initial set up and importing of libraries

In [12]:
%load_ext autoreload
%autoreload 2
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set path to this repository

In [13]:
os.chdir('/home/sapcm15/gitlab/cnbpy')

### Import code from cnbpy library

In [14]:
from cnbpy.fmriprep_cubric import FMRIPREP

### List subjects in folder

List the subjects in the WAND BIDS folder and write to a file called subject_list.txt in another folder (e.g. `/home/sapcm15/gitlab/cnbpy/cnbpy/test/data/subject_list_cubric.txt`)

In [4]:
data_path = "/cubric/collab/314_wand/RS_fMRI_PhysioToolbox"
file_path = "/home/sapcm15/gitlab/cnbpy/cnbpy/test/data/subject_list_cubric.txt"
dir_list = os.listdir(data_path)
file = open(file_path, 'w')
for d in dir_list:    
    if d.startswith("314_"):
        sub = d.replace("314_", "sub-")
        sub = sub.replace("_314WAND_rsfMRI", "")
        sub = sub.replace("_314WANS_rsfMRI", "")
        #print(sub)
        file.write(sub + "\n")
file.close()

### Set up job scripts
First, create an object called `myfmriprep` that loads the `FMRIPREP` code and provides a name for the SLURM job - in this case "WAND_fmriprep"

In [15]:
myfmriprep=FMRIPREP(jobname='WAND_fmriprep')

Use the `populate_all` function from `FMRIPREP` to populate the fields in `template_cubric.sh` using the dictionary provided in `fmriprep_config_cubric.yml`

In [16]:
myfmriprep.populate_all()

Write the new, populated SLURM code to a text file called `WAND_fmriprep_allsubs.sh` or whatever you defined as the job name in the line above. The job file will be saved to the location you defined as the `job_path` in `fmriprep_config_cubric.yml`, e.g., ` job_path: '/cubric/newscratch/314_wand/temp_cm/fmriprep_test/derivatives/fmriprep/jobs'`. 

In [17]:
myfmriprep.writeout()

### Run on the cluster
Use the `execute` function from `FMRIPREP` to send the SLURM job file to the cluster. The error and output files will be written to the same location as the job file. The output of fMRIprep will be written to the location you defined as `outpath_base` in `fmriprep_config_cubric.yml`, e.g.,  `outpath_base: '/cubric/newscratch/314_wand/temp_cm/fmriprep_test/derivatives/fmriprep/preprocessed'`.

In [19]:
myfmriprep.execute()

job WAND_fmriprep_allsubs sent to slurm
You can view in the queue via squeue -u (my username)
